# Imports

In [1]:
from DbConnector import DbConnector
import pandas as pd
import numpy as np
import geopy.distance
from tqdm import tqdm
import datetime as dt

# Preparation

In [2]:
connection = DbConnector()
client = connection.client
db = connection.db

You are connected to the database: third_ass
-----------------------------------------------



# 1. How many users, activities and trackpoints are there in the dataset (after it is inserted into the database).

In [3]:
collections = ["User", "Activity", "TrackPoint"]

for collection in collections:
	amount_of_documents = db[collection].count()
	print(f"amount of {collection.lower()}s in collection {collection}: {amount_of_documents}")

amount of users in collection User: 182
amount of activitys in collection Activity: 22482
amount of trackpoints in collection TrackPoint: 4034966


<ipython-input-3-c750d0e73239>:4: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  amount_of_documents = db[collection].count()


# 2. Find the average number of activities per user.

In [4]:
amount_of_users = db["User"].count()
amount_of_activities = db["Activity"].count()

print(f"Answer: average number of activities per user: {(amount_of_activities/amount_of_users):.3f}")

average number of activities per user: 123.527


<ipython-input-4-e4b0ba46b170>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  amount_of_users = db["User"].count()
<ipython-input-4-e4b0ba46b170>:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  amount_of_activities = db["Activity"].count()


# 3. Find the top 20 users with the highest number of activities.

In [5]:
result = db.Activity.aggregate([{
  "$group": {
    "_id": "$user_id",
    "count": {
      "$sum": 1
    }
  }
},{
  "$sort": {
    "count": -1
  }
},{
  "$limit": 20
},{
  "$project": {
    "user_id": "$_id",
    "count": 1,
    "_id": 0
  }
}])

df = pd.DataFrame(result).rename(columns={"count": "amount_of_activities"})

In [6]:
i = 0
for _, row in df.iterrows():
    print(f"{i+1}. id = {row[1]} (amount of activities = {row[0]})")
    i += 1

1. id = 163 (amount of activities = 3182)
2. id = 085 (amount of activities = 1298)
3. id = 153 (amount of activities = 1123)
4. id = 068 (amount of activities = 969)
5. id = 167 (amount of activities = 944)
6. id = 128 (amount of activities = 937)
7. id = 062 (amount of activities = 782)
8. id = 025 (amount of activities = 757)
9. id = 041 (amount of activities = 557)
10. id = 075 (amount of activities = 509)
11. id = 126 (amount of activities = 468)
12. id = 010 (amount of activities = 434)
13. id = 052 (amount of activities = 425)
14. id = 084 (amount of activities = 422)
15. id = 004 (amount of activities = 395)
16. id = 017 (amount of activities = 391)
17. id = 140 (amount of activities = 380)
18. id = 003 (amount of activities = 322)
19. id = 179 (amount of activities = 319)
20. id = 030 (amount of activities = 296)


# 4. Find all users who have taken a taxi.

In [7]:
df = pd.DataFrame(db.Activity.find({
  "transportation_mode": "taxi"
} , {
  "_id": 0,
  "user_id": 1
})).drop_duplicates()

i = 0
for _, row in df.iterrows():
  print(f"{i+1}: id {row['user_id']} took a taxi")
  i += 1

1: id 010 took a taxi
2: id 020 took a taxi
3: id 021 took a taxi
4: id 052 took a taxi
5: id 056 took a taxi
6: id 058 took a taxi
7: id 062 took a taxi
8: id 065 took a taxi
9: id 068 took a taxi
10: id 075 took a taxi
11: id 078 took a taxi
12: id 080 took a taxi
13: id 082 took a taxi
14: id 084 took a taxi
15: id 085 took a taxi
16: id 091 took a taxi
17: id 098 took a taxi
18: id 100 took a taxi
19: id 102 took a taxi
20: id 104 took a taxi
21: id 105 took a taxi
22: id 111 took a taxi
23: id 114 took a taxi
24: id 118 took a taxi
25: id 126 took a taxi
26: id 128 took a taxi
27: id 139 took a taxi
28: id 147 took a taxi
29: id 153 took a taxi
30: id 154 took a taxi
31: id 161 took a taxi
32: id 163 took a taxi
33: id 167 took a taxi
34: id 175 took a taxi
35: id 179 took a taxi


# 5. Find all types of transportation modes and count how many activities that are tagged with these transportation mode labels. Do not count the rows where the mode is null.

In [8]:
df = pd.DataFrame(db.Activity.aggregate([{
  "$match": {
    "transportation_mode": {
      "$exists": True
    }
  }
},{
  "$group": {
    "_id": "$transportation_mode",
    "count": {
      "$sum": 1
    }
  }
},{
  "$project": {
    "transportation_mode": "$_id",
    "count": 1,
    "_id": 0
  }
}]))

df = df[df.transportation_mode.notnull()]

i = 1
for _, row in df.iterrows():
    print(f"{i}: {row[1]} -> {row[0]} times taken")
    i += 1

1: boat -> 7 times taken
2: subway -> 813 times taken
3: walk -> 6460 times taken
4: bike -> 2089 times taken
5: run -> 6 times taken
6: motorcycle -> 2 times taken
7: bus -> 2853 times taken
8: taxi -> 1179 times taken
9: train -> 299 times taken
10: car -> 993 times taken
11: airplane -> 17 times taken


# 6.1. Find the year with the most activities.

In [9]:
collection = db["Activity"]
documents = collection.find({})
df = pd.DataFrame(documents)
df2 = pd.DataFrame(df.groupby(by=df.start_date_time.dt.year).size().sort_values(ascending=False)).head(1).reset_index().rename(columns={0: "amount_of_activities", "start_date_time": "year"})

for i, row in df2.iterrows():
    print(f"{i+1}: {row[0]} was the year with the most activities ({row[1]}).")

1: 2008 was the year with the most activities (10943).


# 6.2. Is this also the year with most recorded hours?

In [10]:
collection = db["Activity"]
documents = collection.find({})
df = pd.DataFrame(documents)

data = {"id": df["_id"],"year": df["start_date_time"].dt.year, "start": df["start_date_time"], "end": df["end_date_time"]}

# df2 = pd.DataFrame(data).sort_values(by=["date"], ascending=True)
df2 = pd.DataFrame(data)
df2["amount_of_hours"] = None
for i in tqdm(range(df2.shape[0])):
    df2.at[i, "amount_of_hours"] = (df2.at[i, "end"] - df2.at[i, "start"]).total_seconds() / 3600


print(f'The year with the most recorded hours is {df2.drop(["id", "start", "end"], axis=1).groupby(by=["year"]).sum().sort_values(by=["amount_of_hours"], ascending=False).head(1).index[0]}.')
df2.drop(["id", "start", "end"], axis=1).groupby(by=["year"]).sum()

100%|██████████| 22482/22482 [00:00<00:00, 25269.81it/s]

The year with the most recorded hours is 2009.


,amount_of_hours
year,
2007,2950.783056
2008,13953.443611
2009,21805.420833
2010,658.914444
2011,1023.503889
2012,338.660833


# 7. Find the total distance (in km) walked in 2008, by user with id=112.

In [11]:
collection_activity = db["Activity"]
documents_act = collection_activity.find({})
df_activity = pd.DataFrame(documents_act)

collection_trackpoint = db["TrackPoint"]
documents_track = collection_trackpoint.find({})
df_trackpoint = pd.DataFrame(documents_track)

df_activity = df_activity.rename(columns={"_id": "activity_id"})

df = df_activity.merge(df_trackpoint, left_on="activity_id", right_on="activity_id")
df.head()

,activity_id,user_id,transportation_mode,start_date_time,end_date_time,_id,lat,lon,altitude,date_days,date_time
0,634971c3ff1dc6a01d543e86,010,train,2008-03-28 16:00:00,2008-03-28 22:02:00,63497300ff1dc6a01d82f4e3,39.50293,116.714948,-777,39535.6666782407,2008-03-28 16:00:01
1,634971c3ff1dc6a01d543e86,010,train,2008-03-28 16:00:00,2008-03-28 22:02:00,63497300ff1dc6a01d82f4e4,39.497045,116.726137,-777,39535.6673611111,2008-03-28 16:01:00
2,634971c3ff1dc6a01d543e86,010,train,2008-03-28 16:00:00,2008-03-28 22:02:00,63497300ff1dc6a01d82f4e5,39.489695,116.740047,-777,39535.6680439815,2008-03-28 16:01:59
3,634971c3ff1dc6a01d543e86,010,train,2008-03-28 16:00:00,2008-03-28 22:02:00,63497300ff1dc6a01d82f4e6,39.481438,116.755648,-777,39535.6687384259,2008-03-28 16:02:59
4,634971c3ff1dc6a01d543e86,010,train,2008-03-28 16:00:00,2008-03-28 22:02:00,63497300ff1dc6a01d82f4e7,39.472748,116.770972,-777,39535.6694212963,2008-03-28 16:03:58


In [12]:
df = df[(df["user_id"] == "112") & (df["transportation_mode"] == "walk")][["lat", "lon", "activity_id"]]

total_distance = 0

for i in tqdm(range(df.shape[0] - 1)):    
    if df.iloc[i]["activity_id"] == df.iloc[i+1]["activity_id"]:
      total_distance += geopy.distance.geodesic((df.iloc[i]["lat"], df.iloc[i]["lon"]),  (df.iloc[i+1]["lat"], df.iloc[i+1]["lon"])).km

print(f"The total disctance walked by user with id 112 in 2008 is {total_distance} km.")

100%|██████████| 21809/21809 [00:11<00:00, 1877.31it/s]

The total disctance walked by user with id 112 in 2008 is 109.15471726077064 km.


# 8. Find the top 20 users who have gained the most altitude meters. - Is wrong
- Output should be a field with (id, total meters gained per user).
- Remember that some altitude-values are invalid
- Tip: ∑ (𝑡𝑝 𝑛.𝑎𝑙𝑡𝑖𝑡𝑢𝑑𝑒−𝑡𝑝 𝑛−1.𝑎𝑙𝑡𝑖𝑡𝑢𝑑𝑒), 𝑡𝑝 𝑛. 𝑎𝑙𝑡𝑖𝑡𝑢𝑑𝑒 > 𝑡𝑝 𝑛−1.𝑎𝑙𝑡𝑖𝑡𝑢𝑑𝑒

In [13]:
collection_activity = db["Activity"]
documents_act = collection_activity.find({}, {"user_id": 1})
df_activity = pd.DataFrame(documents_act).sort_values(by=["user_id"], ascending=True)

collection_trackpoint = db["TrackPoint"]
documents_track = collection_trackpoint.find({"altitude": {"$ne": -777}}, {"activity_id": 1, "altitude": 1, "_id": 0})
df_trackpoint = pd.DataFrame(documents_track)

df_activity = df_activity.rename(columns={"_id": "activity_id"})

df = df_activity.merge(df_trackpoint, left_on="activity_id", right_on="activity_id").drop(["activity_id"], axis=1)
df.head()

,user_id,altitude
0,000,158
1,000,187
2,000,185
3,000,191
4,000,185


In [14]:
df["altitude"] = df["altitude"].astype(float)
df["meter_gained"] = df.groupby(by=["user_id"]).diff()

df = df.groupby(by=["user_id"]).sum().sort_values(by=["meter_gained"], ascending=False).head(20).reset_index()
df.drop(["altitude"], axis=1, inplace=True)
df.rename(columns={"user_id": "id", "meter_gained": "total meters gained per user"}, inplace=True)
print("20 users who have gained the most altitude meters:")
df

20 users who have gained the most altitude meters:


,id,total meters gained per user
0,166,8494.100000
1,042,3333.300000
2,099,1499.343832
3,144,1456.700000
4,024,1439.000000
5,103,1394.300000
6,139,954.724409
7,123,800.274724
8,060,698.818898
9,066,567.585302


# 9. Find all users who have invalid activities, and the number of invalid activities per user - Amount of activities is wrong, user is right. Maybe difference between python and sql in how the handle diff in datetime
- An invalid activity is defined as an activity with consecutive trackpoints where the timestamps deviate with at least 5 minutes.

In [15]:
users = db.User.find({})
df_us = pd.DataFrame(users)

activities = db.Activity.find({})
df_acts = pd.DataFrame(activities)

trackpoints = db.TrackPoint.find({})
df_tps = pd.DataFrame(trackpoints)

In [16]:
merged = df_acts.rename(columns={"_id": "activity_id"}).merge(df_tps, on="activity_id")

In [17]:
df_trackpoints = merged

df_trackpoints["diff_in_time"] = None
for i in tqdm(range(df_trackpoints.shape[0])):
    if i == 0:
        prev_time = df_trackpoints.at[i, "date_time"]
        continue
    df_trackpoints.at[i, "diff_in_time"] = (df_trackpoints.at[i, "date_time"] - prev_time).total_seconds()
    prev_time = df_trackpoints.at[i, "date_time"]

df_trackpoints

100%|██████████| 4034966/4034966 [04:03<00:00, 16600.55it/s]


,activity_id,user_id,transportation_mode,start_date_time,end_date_time,_id,lat,lon,altitude,date_days,date_time,diff_in_time
0,634971c3ff1dc6a01d543e86,010,train,2008-03-28 16:00:00,2008-03-28 22:02:00,63497300ff1dc6a01d82f4e3,39.50293,116.714948,-777,39535.6666782407,2008-03-28 16:00:01,None
1,634971c3ff1dc6a01d543e86,010,train,2008-03-28 16:00:00,2008-03-28 22:02:00,63497300ff1dc6a01d82f4e4,39.497045,116.726137,-777,39535.6673611111,2008-03-28 16:01:00,59.0
2,634971c3ff1dc6a01d543e86,010,train,2008-03-28 16:00:00,2008-03-28 22:02:00,63497300ff1dc6a01d82f4e5,39.489695,116.740047,-777,39535.6680439815,2008-03-28 16:01:59,59.0
3,634971c3ff1dc6a01d543e86,010,train,2008-03-28 16:00:00,2008-03-28 22:02:00,63497300ff1dc6a01d82f4e6,39.481438,116.755648,-777,39535.6687384259,2008-03-28 16:02:59,60.0
4,634971c3ff1dc6a01d543e86,010,train,2008-03-28 16:00:00,2008-03-28 22:02:00,63497300ff1dc6a01d82f4e7,39.472748,116.770972,-777,39535.6694212963,2008-03-28 16:03:58,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4034961,634971c3ff1dc6a01d549655,171,None,2007-07-23 03:31:53,2007-07-23 10:06:38,6349735cff1dc6a01d9227e7,39.9738166666667,116.324116666667,265.748031496063,39286.4182523148,2007-07-23 10:02:17,116.0
4034962,634971c3ff1dc6a01d549655,171,None,2007-07-23 03:31:53,2007-07-23 10:06:38,6349735cff1dc6a01d9227e8,39.9733333333333,116.323683333333,265.748031496063,39286.4190277778,2007-07-23 10:03:24,67.0
4034963,634971c3ff1dc6a01d549655,171,None,2007-07-23 03:31:53,2007-07-23 10:06:38,6349735cff1dc6a01d9227e9,39.9737333333333,116.32325,364.173228346457,39286.4194212963,2007-07-23 10:03:58,34.0
4034964,634971c3ff1dc6a01d549655,171,None,2007-07-23 03:31:53,2007-07-23 10:06:38,6349735cff1dc6a01d9227ea,39.9733166666667,116.322966666667,246.062992125984,39286.4202430556,2007-07-23 10:05:09,71.0


In [18]:
users_invalid_act = df_trackpoints[df_trackpoints["diff_in_time"] >= 300]
# pd.DataFrame(users_invalid_act.groupby(by=["user_id"]).size())
df.drop_duplicates(keep="first", inplace=True)
dict = users_invalid_act.groupby(by=["user_id"]).count().reset_index().set_index('user_id').T.to_dict('list')

print("-------------------------------------------------------")
print(f"There are {len(dict)} users with invalid activities")
print("-------------------------------------------------------")

for id in dict:
    print(f"user with id {id} has {dict[id][0]} invalid activities.")

-------------------------------------------------------
There are 164 users with invalid activities
-------------------------------------------------------
user with id 000 has 314 invalid activities.
user with id 001 has 87 invalid activities.
user with id 002 has 255 invalid activities.
user with id 003 has 623 invalid activities.
user with id 004 has 883 invalid activities.
user with id 005 has 110 invalid activities.
user with id 006 has 35 invalid activities.
user with id 007 has 70 invalid activities.
user with id 008 has 24 invalid activities.
user with id 009 has 80 invalid activities.
user with id 010 has 134 invalid activities.
user with id 011 has 92 invalid activities.
user with id 012 has 111 invalid activities.
user with id 013 has 82 invalid activities.
user with id 014 has 258 invalid activities.
user with id 015 has 79 invalid activities.
user with id 016 has 48 invalid activities.
user with id 017 has 429 invalid activities.
user with id 018 has 68 invalid activities.

# 10. Find the users who have tracked an activity in the Forbidden City of Beijing.
- In this question you can consider the Forbidden City to have coordinates that correspond to: lat 39.916, lon 116.397.

In [26]:
activities = db.Activity.find({}, {"_id", "user_id"})
df_acts = pd.DataFrame(activities).rename(columns={"_id": "activity_id"})

trackpoints = db.TrackPoint.find({}, {"lat", "lon", "activity_id"})
df_tps = pd.DataFrame(trackpoints)

backup = df_acts.merge(df_tps, on="activity_id")
backup.head()

,activity_id,user_id,_id,lat,lon
0,634971c3ff1dc6a01d543e86,010,63497300ff1dc6a01d82f4e3,39.50293,116.714948
1,634971c3ff1dc6a01d543e86,010,63497300ff1dc6a01d82f4e4,39.497045,116.726137
2,634971c3ff1dc6a01d543e86,010,63497300ff1dc6a01d82f4e5,39.489695,116.740047
3,634971c3ff1dc6a01d543e86,010,63497300ff1dc6a01d82f4e6,39.481438,116.755648
4,634971c3ff1dc6a01d543e86,010,63497300ff1dc6a01d82f4e7,39.472748,116.770972


In [28]:
df = backup

users_in_forbidden_area = df[((df["lat"].apply(lambda x: float(x)) < 39.917) & (df["lat"].apply(lambda x: float(x)) > 39.915)) & ((df["lon"].apply(lambda x: float(x)) < 116.398) & (df["lon"].apply(lambda x: float(x)) > 116.396))].drop_duplicates(subset=["user_id"]).reset_index()["user_id"]
for i, id in enumerate(users_in_forbidden_area.sort_values(ascending=True)):
    print(f"{i}. {id} was in the forbidden area.")

0. 004 was in the forbidden area.
1. 018 was in the forbidden area.


# 11. Find all users who have registered transportation_mode and their most used transportation_mode.
- The answer should be on format (user_id, most_used_transportation_mode) sorted on user_id.
- Some users may have the same number of activities tagged with e.g. walk and car. In this case it is up to you to decide which transportation mode to include in your answer (choose one).
- Do not count the rows where the mode is null.

In [30]:
users = db.User.find({"has_labels": True} , {"has_labels": 1})
df_us = pd.DataFrame(users).rename(columns={"_id": "user_id"})

activities = db.Activity.find({}, {"transportation_mode": 1, "user_id": 1})
df_acts = pd.DataFrame(activities)

backup = df_us.merge(df_acts, on="user_id")
backup.head()

,user_id,has_labels,_id,transportation_mode
0,104,True,634971c3ff1dc6a01d5457c5,bus
1,104,True,634971c3ff1dc6a01d5457c6,bus
2,104,True,634971c3ff1dc6a01d5457c7,bus
3,104,True,634971c3ff1dc6a01d5457c8,bus
4,104,True,634971c3ff1dc6a01d5457c9,car


In [33]:
df = backup
df = df.drop(["has_labels", "_id"], axis=1)
df2 = pd.DataFrame(df.groupby(by=["user_id", "transportation_mode"]).size())
df3 = df2.reset_index().rename(columns={0: "times_used"}).sort_values(by=["user_id", "transportation_mode"], ascending=[True, False]).drop_duplicates(subset=["user_id"], keep="first")

i = 0
for _, row in df3.iterrows():
    print(f"{i+1}: user with id {row[0]} most used transportation_mode is {row[1]} ({row[2]} times)")
    i += 1

1: user with id 010 most used transportation_mode is walk (153 times)
2: user with id 020 most used transportation_mode is walk (76 times)
3: user with id 021 most used transportation_mode is walk (7 times)
4: user with id 052 most used transportation_mode is walk (131 times)
5: user with id 053 most used transportation_mode is walk (14 times)
6: user with id 056 most used transportation_mode is walk (6 times)
7: user with id 058 most used transportation_mode is walk (11 times)
8: user with id 059 most used transportation_mode is walk (1 times)
9: user with id 060 most used transportation_mode is walk (2 times)
10: user with id 062 most used transportation_mode is walk (324 times)
11: user with id 064 most used transportation_mode is walk (24 times)
12: user with id 065 most used transportation_mode is walk (63 times)
13: user with id 067 most used transportation_mode is walk (63 times)
14: user with id 068 most used transportation_mode is walk (128 times)
15: user with id 069 most use